In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try: 
    credential = DefaultAzureCredential
    #check in given credential can get token seccessfully.
    credential.get_token("https://manegement.azure.com/.default")
except:
    # Fall Back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential 

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
%%writefile src/sorvetes-training.py
# import libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the sorvetes dataset
print("Loading Data... ")
sorvetes = pd.read_csv('sorvetes.csv')

# separete features and labels

X, y = sorvetes[['Temperatura']].values, sorvetes['Vendas'].values

#split data into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

#set regularization hyperparameter

reg= 0.01

#train a logistic regression model

print('Training a logistic regression model with regularization rate of', reg) 
model = LogisticRegression(C=1/reg, solver = "liblinear").fit(X_train, y_train)

#calculate accuracy

y_hat = model.predict(X_test) 
acc = np.average(y_hat == y_test) 
print('Accuracy:', acc)

#calculate NIC

y_scores = model.predict_proba(X_test) 
auc = roc_auc_score(y_test,y_scores [:,1]) 
print('AUC: ' + str(auc))